<a href="https://colab.research.google.com/github/scadusseau/Planification/blob/master/LRTDP_and_EP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LRTDP

Those code lines correspond to my implementation of B. Bonet and H. Geffner algorithm LRTDP. The way to use this algoritm is very similar to mdptoolbox dependency (similar variables and function declaration). 

#Imports

In [ ]:
import numpy as np # used for vectorial calculus
import time # this library is mostly used to print values, as a debugger

#LRTDP

In [ ]:
#some basic functions to compute the problem's dimensions.
#those functions are very simple, and it was not necessary to implement them explicitly.
def compute_states(C): # computes the number of states
  return len(C)

def compute_actions(C): # compute the number of actions
  return len(C[0])


#Before we can run LRTDP, we need to define the environment's shape, that will be the same for different algorithms.
class mdp:
  """
 - T is the transitions matrix for each action. This variable is defined by the user, in the same way that it is defined with mdptoolbox.
   It is np array defined as T = np.array([Action1, Action2, ...]) and each action is a transition matrix defined as :
            
                 state1 state2 ...
          state1  p11    p12   ...
          state2  p21    p22   ...
            .      .      .    ...
Action1 =   .      .      .    ...
            .      .      .    ...

where p is the probability to reach a state B from a state A doing the considered Action
and where the sum of all the probabilities of a same line is equal to 1.

  - C is the costs matrix and is also similar to mdptoolbox Rewards matrix. It is another np array giving the cost of reaching a given state doing a given action.
  It is defined as :

                Action1 Action2 ...
        state1    c11     c12   ...
        state2    c21     c22   ...
          .        .       .    ...
Costs =   .        .       .    ...
          .        .       .    ...

where c <= 0

  - gamma is the discount factor, and is defined by the user.

  - epsilon is a precision criterion for LRTDP once again defined by the user.

  - nb_actions is the number of actions.

  - nb_states is the number of states.

  - V is the V-function np array. It is initialized with 0 for each states, and then the values are updated by the algorithm.

  - solved is a list containing solved states. If a state is not in this list it is equivalent to state.solved == False.

  - s is an integer that represents the current state. Initial state must be 0.

  - verbosity is a boolean. When it is True, V is displayed in the console, each time it is updated.

  We can notice that, as in mdptoolbox, states are not explicitly defined. The states features are actually defined in other variables such as T, V and solved.

  """

  def __init__(self, T, C, gamma, epsilon, verbosity, mod):
    self.mod = mod
    self.T = T
    self.costs = C
    self.gamma = gamma
    self.epsilon = epsilon
    self.nb_actions = compute_actions(C)
    self.nb_states = compute_states(C)
    self.V = np.zeros(self.nb_states)
    self.solved = []
    self.s = 0
    self.policy = np.zeros(self.nb_states)

    if verbosity == None:
      self.verbosity = False
    
    else:
      self.verbosity = verbosity

  
  #Basic functions
  def Q_Value(self, a):
    if self.mod == 1:
      return self.costs[self.s][a] + self.gamma * np.sum((self.T[a][self.s][:] * self.V[:]))

    elif self.mod == 0:
      vect = []
      for i in range(len(self.T[a][self.s][:])):
        if self.T[a][self.s][i] > 0:
          vect.append(self.V[i])
      return self.costs[self.s][a] + self.gamma * max(vect[:])


  def greedyAction(self):
    Q = np.zeros(self.nb_actions)
    
    for i in range(self.nb_actions):
      Q[i] = self.Q_Value(i)

    #time.sleep(0.1)
    #print("state")
    #print(self.s)
    #print("##############")
    #print(Q)
    #print(np.argmax(Q))
    return np.argmax(Q) #here we look for the max argument, because the costs are negative
  
  def update(self):
    if self.verbosity:
      print("Value-functions:")
      print(self.V)
      print("Policy:")
      print(self.policy)

    a = self.greedyAction()
    self.policy[self.s] = a
    # stocker a dans le vecteur policy.
    self.V[self.s] = self.Q_Value(a)

  def pickNextState(self, a):
    candidates = [i for i in range(0, self.nb_states)]
    return np.random.choice(candidates, p=self.T[a][self.s])

  def residual(self):
    a = self.greedyAction()
    return np.abs(self.Q_Value(a) - self.V[self.s])

  
  
  def checkSolved(self):
    rv = True
    opened = []  #original name is open, but in python open refers to a function
    closed = []
  
    if (self.s not in self.solved):
      opened.append(self.s)
  
    while len(opened) != 0:
      self.s = opened.pop()
      closed.append(self.s)

      if self.residual() > self.epsilon:
        rv = False   
      
      #print("\nCheckSolved")
      #print("residual:")
      #print(self.residual())
      #print("state:")
      #print(self.s)
      #print("")
      
      a = self.greedyAction()

      for i in range(self.nb_states):
        if self.T[a][self.s][i] != 0:         
          if (i not in self.solved) and (i not in opened + closed):
            opened.append(i)
    #print("closed")
    #print(closed)
    #print("")
        
    if rv:     
      for j in range(len(closed)):
        self.solved.append(closed[j])
        #print(self.solved)
      
    else:
      #print("closed")
      #print(closed)
      #print("")
      while closed != []:
        self.s = closed.pop()
        self.update()

    return rv


  def LRTDP_TRIAL(self):
    visited = []

    while self.s not in self.solved:
      visited.append(self.s)

      if self.costs[self.s][0] >= 0: #Here we can define a goal state as a state where the reward is not negative
        break
      
      a = self.greedyAction()
      self.update()

      self.s = self.pickNextState(a)
      #print("\nTRIAL")
      #print("action")
      #print(a)
      #print("##############")
      #print("state")
      #print(self.s)
      #print("##############\n")

      while visited != []:
        self.s = visited.pop()
        #print("solved")
        #print(self.solved)
        if not self.checkSolved():
          break
      
  def LRTDP(self):
    #iter = 0
    while self.s not in self.solved:
      #print("\n===============")
      #print(iter)
      #print("===============\n")
      #iter += 1

      #time.sleep(0.1)
      #print("solved")
      #print(self.solved)
      #print("##########")
      self.LRTDP_TRIAL()

    return self.policy, self.V


In [ ]:
"""
TireWorld Problem:
A car has to reach the goal state, and can chose different roads. Some roads can lead to dead-ends -the car cannot advance anymore-, some road are safe, but increase the distance
to the goal. The goal of this exercise is to find balance between safety and speed.
The probability to reach a dead-end from an intermediate state is 0.5 when a dead end is directly linked to this state, for any action chosen. Else, the probability to reach a
non dead end state from another non dead state (that is not linked to a dead end state) is 1.0 (The probability to do an action and reach the expected state is 1.0).
The cost of the road between two states is 1.0

Actions: North, East, West (NEW)
[1. 2. 2. 2. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 1. 0. 0. 1. 0. 0. 2. 1. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0.]
19 states: 13 intermediate states, 4 dead ends {14, 15, 16, 17}, 1 initial state {S}, 1 Goal state {G}

North:  <--- 
East: \
West: /

             13
            /   \
           /     \
          /  17   \
         12   ^   11
        /   \ | /   \
       /     \|/     \
      10 <--- 9  <--- 8
     /  \   /   \   /  \
    /    \ /     \ /    \
   7      6       5      4
  /  \   /  \   /  \   /  \
 /    \ /    \ /    \ /    \
G <--- 3 <--- 2 <--- 1 <--- S(0)
       |      |      |
       |      |      |
      16     15      14

"""

"""
North:
     S     1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16   17   G
S   [0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
1   [0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.],
2   [0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.],
3   [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0., 0.5],
4   [0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
5   [0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
6   [0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
7   [0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
8   [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
9   [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.],
10  [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
11  [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
12  [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.],
13  [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.],
14  [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.],
15  [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.],
16  [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
17  [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.],
G   [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0],
"""

N = [[0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.],
     [0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0., 0.5],
     [0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0]]


# East
#     S     1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16   17   G
E = [[0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0., 0.5,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0]]


# West
#     S     1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16   17   G
W = [[1.0, 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0],
     [0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0]]



T = np.array([N,E,W])








N = [[0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.],
     [0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0., 0.5],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0]]


# East
#     S     1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16   17   G
E = [[0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0., 0.5,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0]]


# West
#     S     1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16   17   G
W = [[0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0],
     [0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0]]



T = np.array([N,E,W])

# Costs/Rewards -> columns are related to actions
# Let's first define rewards so that the agent recieves a strong negative reward when it reaches a dead end state
C = np.array([[-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-10.00, -10.00, -10.00],
              [-10.00, -10.00, -10.00],
              [-10.00, -10.00, -10.00],
              [-10.00, -10.00, -10.00],
              [0.00,  0.00,  0.00]])

# Costs/Rewards -> columns are related to actions
# Let's first define rewards so that the agent recieves a strong negative reward when it reaches a dead end state
X = np.array([[-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [0.00,  0.00,  0.00]])



In [ ]:
# MDP MAZE EXAMPLE
#####################
#[N,S,E,W]
#[2. 2. 2. 0. 0. 3. 0. 0. 3. 3. 1.]
#[2. 2. 2. 0. 0. 0. 0. 1. 3. 1. 1.]
# --------------
# |s0 s1 s2 s3+ |
# |s4  # s5 S6- |
# |s7 s8 s9 s10 |
# --------------
# The + indicates a reward of 1.0, the - a penalty of -1.0.
# The # in the middle of the maze is an obstruction.
# Rewards and penalties are associated with states, not actions.
# The default reward/penalty is -0.04.
# There is no discounting, but a there is an absorbing state that
# + and - transition to automatically.  The absorbing state cannot be exited.
# 
# 
# The actions, NSEW, have the expected result 80% of the time, and
# transition in a direction perpendicular to the intended on with a 10%
# probability for each direction.  Movement into a wall returns the agent
# to its original state.
# 
# good = +1 reward, bad = -1 penalty,

# north
"""
P(:,:,1) = [ 0.9 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
             0.1 0.8 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
             0.0 0.1 0.8 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
             0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
             0.8 0.0 0.0 0.0 0.2 0.0 0.0 0.0 0.0 0.0 0.0;
             0.0 0.0 0.8 0.0 0.0 0.1 0.1 0.0 0.0 0.0 0.0;
             0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0;
             0.0 0.0 0.0 0.0 0.8 0.0 0.0 0.1 0.1 0.0 0.0;
             0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.1 0.8 0.1 0.0;
             0.0 0.0 0.0 0.0 0.0 0.8 0.0 0.0 0.1 0.0 0.1;
             0.0 0.0 0.0 0.0 0.0 0.0 0.8 0.0 0.0 0.1 0.1];
"""
N = [[ 0.9, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.1, 0.8, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.1, 0.8, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.8, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.8, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.8, 0.1, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.1, 0.0, 0.1],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.1, 0.1]]

# east
"""
P(:,:,2) = [0.1 0.8 0.0 0.0 0.1 0.0 0.0 0.0 0.0 0.0 0.0;
            0.0 0.2 0.8 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
            0.0 0.0 0.1 0.8 0.0 0.1 0.0 0.0 0.0 0.0 0.0;
            0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
            0.1 0.0 0.0 0.0 0.8 0.0 0.0 0.1 0.0 0.0 0.0;
            0.0 0.0 0.1 0.0 0.0 0.0 0.8 0.0 0.0 0.1 0.0;
            0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0;
            0.0 0.0 0.0 0.0 0.1 0.0 0.0 0.1 0.8 0.0 0.0;
            0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.2 0.8 0.0;
            0.0 0.0 0.0 0.0 0.0 0.1 0.0 0.0 0.1 0.0 0.8;
            0.0 0.0 0.0 0.0 0.0 0.0 0.1 0.0 0.0 0.0 0.9];
"""
E = [[ 0.1, 0.8, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.2, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.1, 0.8, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.1, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.1, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.1, 0.8, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.8, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.1, 0.0, 0.8],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.9]]

# south    
"""
P(:,:,3) = [ 0.1 0.1 0.0 0.0 0.8 0.0 0.0 0.0 0.0 0.0 0.0;
             0.1 0.8 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
             0.0 0.1 0.0 0.1 0.0 0.8 0.0 0.0 0.0 0.0 0.0;
             0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
             0.0 0.0 0.0 0.0 0.2 0.0 0.0 0.8 0.0 0.0 0.0;
             0.0 0.0 0.0 0.0 0.0 0.1 0.1 0.0 0.0 0.8 0.0;
             0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0;
             0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.9 0.1 0.0 0.0;
             0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.1 0.8 0.1 0.0;
             0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.1 0.8 0.1;
             0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.1 0.9];
"""
S = [[ 0.1, 0.1, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.1, 0.8, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.1, 0.0, 0.1, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0, 0.8, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9, 0.1, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.8, 0.1, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.8, 0.1],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.9]]
# west
"""
P(:,:,4) = [0.9 0.0 0.0 0.0 0.1 0.0 0.0 0.0 0.0 0.0 0.0;
            0.8 0.2 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
            0.0 0.8 0.1 0.0 0.0 0.1 0.0 0.0 0.0 0.0 0.0;
            0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
            0.1 0.0 0.0 0.0 0.8 0.0 0.0 0.1 0.0 0.0 0.0;
            0.0 0.0 0.1 0.0 0.0 0.8 0.0 0.0 0.0 0.1 0.0;
            0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0;
            0.0 0.0 0.0 0.0 0.1 0.0 0.0 0.9 0.0 0.0 0.0;
            0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.8 0.2 0.0 0.0;
            0.0 0.0 0.0 0.0 0.0 0.1 0.0 0.0 0.8 0.1 0.0;
            0.0 0.0 0.0 0.0 0.0 0.0 0.1 0.0 0.0 0.8 0.1];
"""     
W = [[ 0.9, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
      [ 0.8, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
      [ 0.0, 0.8, 0.1, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
      [ 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
      [ 0.1, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0],
      [ 0.0, 0.0, 0.1, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.1, 0.0],
      [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
      [ 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.9, 0.0, 0.0, 0.0],
      [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.2, 0.0, 0.0],
      [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.8, 0.1, 0.0],
      [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.8, 0.1]]

P = np.array([N,S,E,W])

# Costs/Rewards -> columns are related to actions
R = np.array([[-0.05, -0.05, -0.05, -0.05],
              [-0.05, -0.05, -0.05, -0.05],
              [-0.05, -0.05, -0.05, -0.05],
              [ 0.00,  0.00,  0.00,  0.00],
              [-0.05, -0.05, -0.05, -0.05],
              [-0.05, -0.05, -0.05, -0.05],
              [-50.00, -50.00, -50.00, -50.00],
              [-0.05, -0.05, -0.05, -0.05],
              [-0.05, -0.05, -0.05, -0.05],
              [-0.05, -0.05, -0.05, -0.05],
              [-0.05, -0.05, -0.05, -0.05]])

# Costs/Rewards -> columns are related to actions
X = np.array([[-0.05, -0.05, -0.05, -0.05],
              [-0.05, -0.05, -0.05, -0.05],
              [-0.05, -0.05, -0.05, -0.05],
              [ 0.00,  0.00,  0.00,  0.00],
              [-0.05, -0.05, -0.05, -0.05],
              [-0.05, -0.05, -0.05, -0.05],
              [-0.05, -0.05, -0.05, -0.05],
              [-0.05, -0.05, -0.05, -0.05],
              [-0.05, -0.05, -0.05, -0.05],
              [-0.05, -0.05, -0.05, -0.05],
              [-0.05, -0.05, -0.05, -0.05]])

#LRTDP policy found for Maze

In [ ]:
lrtdp_test2 = mdp(P, X, 0.95, 0.01, True, 1) # Test LRTPD on maze environment
lrtdp_test2.LRTDP()

#Conclusion :
#LRTDP seems to be working, because the Value function vector approximate the optimal policy.

NameError: ignored

#LRTDP policy found for Triangle Tireworld

In [ ]:
#[2. 2. 2. 0. 0. 3. 0. 0. 3. 3. 1.]
#[1. 2. 2. 0. 1. 2. 1. 2. 1. 0. 2. 1. 2. 2. 0. 0. 0. 0. 0.]
#[0. 0. 0. 0. 1. 1. 2. 2. 0. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0.]
#[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]



In [ ]:
lrtdp_test1 = mdp(T, C, 0.95, 0.01, True, 0) # Test LRTPD on Triangle Tireworld environment
lrtdp_test1.LRTDP()

#Conclusion :
#LRTDP seems to be working, because the Value function vector approximate the optimal policy.
#However there is one problem, the algorithm does not converge.
#This issue could be due to the environments; Stochastic Shortest Path with Dead Ends. This issue is shown in A. Kolobov paper, and he solves this problem with FRET algorithm:
#Find and Revise part of this algorithm is actually LRTDP, and Eliminate Traps is another function that helps the algorithm to deal with dead ends 


Streaming output truncated to the last 5000 lines.
[0. 0. 0. 0. 1. 1. 2. 2. 0. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0.]
Value-functions:
[  -3.709875     -2.8525       -1.95         -1.           -2.8525
   -1.95         -1.95         -1.           -1.95         -1.
   -1.95         -1.95         -2.8525       -3.709875   -189.2532908
 -188.68767453 -188.68767453  -19.5           0.        ]
Policy:
[0. 0. 0. 0. 1. 1. 2. 2. 0. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0.]
Value-functions:
[  -3.709875     -2.8525       -1.95         -1.           -2.8525
   -1.95         -1.95         -1.           -1.95         -1.
   -1.95         -1.95         -2.8525       -3.709875   -189.2532908
 -188.68767453 -188.68767453  -19.5           0.        ]
Policy:
[0. 0. 0. 0. 1. 1. 2. 2. 0. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0.]
Value-functions:
[  -3.709875     -2.8525       -1.95         -1.           -2.8525
   -1.95         -1.95         -1.           -1.95         -1.
   -1.95         -1.95         -2.8525       -3.709875   -1

#Maze LRTDP policy simulation

In [ ]:
policy = [2, 2, 2, 0, 0, 3, 0, 0, 3, 3, 1]
nb_states = compute_states(R)
candidates = [i for i in range(0, nb_states)]
print(candidates)
n_episodes = 1000
success = 0
failure = 0

for i in range(n_episodes):
  if i%10 == 0: 
    print("episode " + str(i))
  s = 0
  while s != 3 and s != 6:
    s = np.random.choice(candidates, p=P[policy[s]][s])
    if s == 3:
      success += 1
    
    if s == 6:
      failure += 1

print("Failure : " + str(failure))
print("Success : " + str(success))

#Results: 100% successfull when the initial state is S0.
#The policy found is 100% safe

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
episode 0
episode 10
episode 20
episode 30
episode 40
episode 50
episode 60
episode 70
episode 80
episode 90
episode 100
episode 110
episode 120
episode 130
episode 140
episode 150
episode 160
episode 170
episode 180
episode 190
episode 200
episode 210
episode 220
episode 230
episode 240
episode 250
episode 260
episode 270
episode 280
episode 290
episode 300
episode 310
episode 320
episode 330
episode 340
episode 350
episode 360
episode 370
episode 380
episode 390
episode 400
episode 410
episode 420
episode 430
episode 440
episode 450
episode 460
episode 470
episode 480
episode 490
episode 500
episode 510
episode 520
episode 530
episode 540
episode 550
episode 560
episode 570
episode 580
episode 590
episode 600
episode 610
episode 620
episode 630
episode 640
episode 650
episode 660
episode 670
episode 680
episode 690
episode 700
episode 710
episode 720
episode 730
episode 740
episode 750
episode 760
episode 770
episode 780
episode 790
episode 800
epis

#Triangle Tireworld LRTDP simulation

In [ ]:
policy = [0, 0, 0, 0, 1, 1, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0]
#[1. 2. 2. 2. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
#[1. 2. 2. 0. 1. 2. 1. 2. 1. 0. 2. 1. 2. 2. 0. 0. 0. 0. 0.] S3P
#[0. 0. 0. 0. 1. 1. 2. 2. 0. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0.] S2P
#


#policy = [1, 1, 0, 0, 1, 0, 0, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0]
nb_states = compute_states(C)
candidates = [i for i in range(0, nb_states)]
print(candidates)
n_episodes = 100000
success = 0
failure = 0
too_slow = 0
for i in range(n_episodes):
  if i%10 == 0: 
    print("episode " + str(i))
  s = 0
  n = 0
  while s < 14:
    n += 1
    s = np.random.choice(candidates, p=T[policy[s]][s])
    
    if n >= 200:
      s = 14
      too_slow += 1
    #print(s)
    if s == 18:
      success += 1
    
    if s > 13 and s < 18 :
      failure += 1
  

print("Failure : " + str(failure))
print("Success : " + str(success))
print("Failure due to slow progression : " + str(too_slow))

#The policy obtained does not allow the agent to finish an episode (ie reach the goal or a dead end)

Streaming output truncated to the last 5000 lines.
episode 50040
episode 50050
episode 50060
episode 50070
episode 50080
episode 50090
episode 50100
episode 50110
episode 50120
episode 50130
episode 50140
episode 50150
episode 50160
episode 50170
episode 50180
episode 50190
episode 50200
episode 50210
episode 50220
episode 50230
episode 50240
episode 50250
episode 50260
episode 50270
episode 50280
episode 50290
episode 50300
episode 50310
episode 50320
episode 50330
episode 50340
episode 50350
episode 50360
episode 50370
episode 50380
episode 50390
episode 50400
episode 50410
episode 50420
episode 50430
episode 50440
episode 50450
episode 50460
episode 50470
episode 50480
episode 50490
episode 50500
episode 50510
episode 50520
episode 50530
episode 50540
episode 50550
episode 50560
episode 50570
episode 50580
episode 50590
episode 50600
episode 50610
episode 50620
episode 50630
episode 50640
episode 50650
episode 50660
episode 50670
episode 50680
episode 50690
episode 50700
episode 507

#Equal Penalty

In [ ]:
#some basic functions to compute the problem's dimensions.
#those functions are very simple, and it was not necessary to implement them explicitly.
def compute_states(C): # computes the number of states
  return len(C)

def compute_actions(C): # compute the number of actions
  return len(C[0])


#Before we can run LRTDP, we need to define the environment's shape, that will be the same for different algorithms.
class mdp:
  """
 - T is the transitions matrix for each action. This variable is defined by the user, in the same way that it is defined with mdptoolbox.
   It is np array defined as T = np.array([Action1, Action2, ...]) and each action is a transition matrix defined as :
            
                 state1 state2 ...
          state1  p11    p12   ...
          state2  p21    p22   ...
            .      .      .    ...
Action1 =   .      .      .    ...
            .      .      .    ...

where p is the probability to reach a state B from a state A doing the considered Action
and where the sum of all the probabilities of a same line is equal to 1.

  - C is the costs matrix and is also similar to mdptoolbox Rewards matrix. It is another np array giving the cost of reaching a given state doing a given action.
  It is defined as :

                Action1 Action2 ...
        state1    c11     c12   ...
        state2    c21     c22   ...
          .        .       .    ...
Costs =   .        .       .    ...
          .        .       .    ...

where c <= 0

  - gamma is the discount factor, and is defined by the user.

  - epsilon is a precision criterion for LRTDP once again defined by the user.

  - nb_actions is the number of actions.

  - nb_states is the number of states.

  - V is the V-function np array. It is initialized with 0 for each states, and then the values are updated by the algorithm.

  - solved is a list containing solved states. If a state is not in this list it is equivalent to state.solved == False.

  - s is an integer that represents the current state. Initial state must be 0.

  - verbosity is a boolean. When it is True, V is displayed in the console, each time it is updated.

  We can notice that, as in mdptoolbox, states are not explicitly defined. The states features are actually defined in other variables such as T, V and solved.

  """

  def __init__(self, T, C, gamma, epsilon, verbosity, mod, K, dead_ends, traps):
    self.traps = traps
    self.mod = mod
    self.T = T
    self.costs = C
    self.gamma = gamma
    self.epsilon = epsilon
    self.nb_actions = compute_actions(C)
    self.nb_states = compute_states(C)
    self.V = np.zeros(self.nb_states)
    self.solved = []
    self.s = 0
    self.policy = [0] * self.nb_states#np.zeros(self.nb_states)
    self.K = K
    self.dead_ends = dead_ends
    self.acc_cost = 0

    if verbosity == None:
      self.verbosity = False
    
    else:
      self.verbosity = verbosity

  
  #Basic functions
  def Q_Value(self, a):
    if self.mod == 1:
      #print("hello mod1")
      if self.K != 0 and self.s in self.dead_ends:  # C is not the reward here.
        #print("yes")0

        #print("State : " + str(self.s) + "     Cost : " + str(self.acc_cost))

        return (self.K - self.acc_cost -self.K + self.costs[self.s][a]) + self.gamma * np.sum((self.T[a][self.s][:] * self.V[:])) # costs = path cost, not action cost.

      else: 
        return self.costs[self.s][a] + self.gamma * np.sum((self.T[a][self.s][:] * self.V[:]))

    elif self.mod == 0:
      #print("hello mod2")
      if self.s in self.traps:
        #print("##########################")
        #print(self.costs[self.s][a])
        return self.costs[self.s][a]

      vect = []
      for i in range(len(self.T[a][self.s][:])):
        if self.T[a][self.s][i] > 0:
          vect.append(self.V[i])
      return self.costs[self.s][a] + self.gamma * max(vect[:])


  def greedyAction(self):
    Q = np.zeros(self.nb_actions)
    
    for i in range(self.nb_actions):
      Q[i] = self.Q_Value(i)

    #time.sleep(0.1)
    #print("state")
    #print(self.s)
    #print("##############")
    #print(Q)
    #print(np.argmax(Q))
    return np.argmax(Q) #here we look for the max argument, because the costs are negative
  
  def update(self):
    if self.verbosity:
      print("Value-functions:")
      print(self.V)
      print("Policy:")
      print(self.policy)

    a = self.greedyAction()
    if self.s == 0:
      self.acc_cost = 0
    else:
      self.acc_cost += self.costs[self.s][a]
    self.policy[self.s] = int(a)
    # stocker a dans le vecteur policy. 
    self.V[self.s] = self.Q_Value(a)
    #print("+++++++++++++")
    #print(self.V[self.s])

  def pickNextState(self, a):
    candidates = [i for i in range(0, self.nb_states)]
    return np.random.choice(candidates, p=self.T[a][self.s])

  def residual(self):
    a = self.greedyAction()
    return np.abs(self.Q_Value(a) - self.V[self.s])

  
  
  def checkSolved(self):
    rv = True
    opened = []  #original name is open, but in python open refers to a function
    closed = []
  
    if (self.s not in self.solved):
      opened.append(self.s)
  
    while len(opened) != 0:
      self.s = opened.pop()
      closed.append(self.s)

      if self.residual() > self.epsilon:
        rv = False   
      
      #print("\nCheckSolved")
      #print("residual:")
      #print(self.residual())
      #print("state:")
      #print(self.s)
      #print("")
      
      a = self.greedyAction()

      for i in range(self.nb_states):
        if self.T[a][self.s][i] != 0:         
          if (i not in self.solved) and (i not in opened + closed):
            opened.append(i)
    #print("closed")
    #print(closed)
    #print("")
        
    if rv:     
      for j in range(len(closed)):
        self.solved.append(closed[j])
        #print(self.solved)
      
    else:
      #print("closed")
      #print(closed)
      #print("")
      while closed != []:
        self.s = closed.pop()
        self.update()

    return rv


  def LRTDP_TRIAL(self):
    visited = []

    while self.s not in self.solved:
      visited.append(self.s)

      if self.s == 0:
        self.acc_cost = 0
      
      else:
        self.acc_cost + self.costs[self.s][a]

      if self.costs[self.s][0] >= 0: #Here we can define a goal state as a state where the reward is not negative
        break
        #return self.costs[self.s][0]

      if self.s in self.traps:
        break
        #return self.costs[self.s][0]
      
      a = self.greedyAction()
      self.update()

      self.s = self.pickNextState(a)
      #print("\nTRIAL")
      #print("action")
      #print(a)
      #print("##############")
      #print("state")
      #print(self.s)
      #print("##############\n")

      while visited != []:
        self.s = visited.pop()
        #print("solved")
        #print(self.solved)
        if not self.checkSolved():
          break
      
  def LRTDP(self):
    #iter = 0
    while self.s not in self.solved:
      #print("\n===============")
      #print(iter)
      #print("===============\n")
      #iter += 1

      #time.sleep(0.1)
      #print("solved")
      #print(self.solved)
      #print("##########")
      self.LRTDP_TRIAL()

    return self.policy, self.V

    





In [ ]:
#some basic functions to compute the problem's dimensions.
#those functions are very simple, and it was not necessary to implement them explicitly.
def compute_states(C): # computes the number of states
  return len(C)

def compute_actions(C): # compute the number of actions
  return len(C[0])


#Before we can run LRTDP, we need to define the environment's shape, that will be the same for different algorithms.
class mdp:
  """
 - T is the transitions matrix for each action. This variable is defined by the user, in the same way that it is defined with mdptoolbox.
   It is np array defined as T = np.array([Action1, Action2, ...]) and each action is a transition matrix defined as :
            
                 state1 state2 ...
          state1  p11    p12   ...
          state2  p21    p22   ...
            .      .      .    ...
Action1 =   .      .      .    ...
            .      .      .    ...

where p is the probability to reach a state B from a state A doing the considered Action
and where the sum of all the probabilities of a same line is equal to 1.

  - C is the costs matrix and is also similar to mdptoolbox Rewards matrix. It is another np array giving the cost of reaching a given state doing a given action.
  It is defined as :

                Action1 Action2 ...
        state1    c11     c12   ...
        state2    c21     c22   ...
          .        .       .    ...
Costs =   .        .       .    ...
          .        .       .    ...

where c <= 0

  - gamma is the discount factor, and is defined by the user.

  - epsilon is a precision criterion for LRTDP once again defined by the user.

  - nb_actions is the number of actions.

  - nb_states is the number of states.

  - V is the V-function np array. It is initialized with 0 for each states, and then the values are updated by the algorithm.

  - solved is a list containing solved states. If a state is not in this list it is equivalent to state.solved == False.

  - s is an integer that represents the current state. Initial state must be 0.

  - verbosity is a boolean. When it is True, V is displayed in the console, each time it is updated.

  We can notice that, as in mdptoolbox, states are not explicitly defined. The states features are actually defined in other variables such as T, V and solved.

  """

  def __init__(self, T, C, gamma, epsilon, verbosity, mod, K, dead_ends, traps):
    self.traps = traps
    self.mod = mod
    self.T = T
    self.costs = C
    self.gamma = gamma
    self.epsilon = epsilon
    self.nb_actions = compute_actions(C)
    self.nb_states = compute_states(C)
    self.V = np.zeros(self.nb_states)
    self.solved = []
    self.s = 0
    self.policy = [0] * self.nb_states#np.zeros(self.nb_states)
    self.K = K
    self.dead_ends = dead_ends
    self.acc_cost = 0

    if verbosity == None:
      self.verbosity = False
    
    else:
      self.verbosity = verbosity

  
  #Basic functions
  def Q_Value(self, a):
    if self.mod == 1:
      #print("hello mod1")
      if self.K != 0 and self.s in self.dead_ends:  # C is not the reward here.
        #print("yes")0

        #print("State : " + str(self.s) + "     Cost : " + str(self.acc_cost))

        return (self.K - self.acc_cost -self.K + self.costs[self.s][a]) + self.gamma * np.sum((self.T[a][self.s][:] * self.V[:])) # costs = path cost, not action cost.

      else: 
        return self.costs[self.s][a] + self.gamma * np.sum((self.T[a][self.s][:] * self.V[:]))

    elif self.mod == 0:
      #print("hello mod2")
      if self.s in self.traps:
        #print("##########################")
        #print(self.costs[self.s][a])
        return self.costs[self.s][a]

      vect = []
      for i in range(len(self.T[a][self.s][:])):
        if self.T[a][self.s][i] > 0:
          vect.append(self.V[i])
      return self.costs[self.s][a] + self.gamma * max(vect[:])


  def greedyAction(self):
    Q = np.zeros(self.nb_actions)
    
    for i in range(self.nb_actions):
      Q[i] = self.Q_Value(i)

    #time.sleep(0.1)
    #print("state")
    #print(self.s)
    #print("##############")
    #print(Q)
    #print(np.argmax(Q))
    return np.argmax(Q) #here we look for the max argument, because the costs are negative
  
  def update(self):
    #print("state : " + str(self.s))
    if self.verbosity:
      print("Value-functions:")
      print(self.V)
      print("Policy:")
      print(self.policy)

    a = self.greedyAction()
    if self.s == 0:
      self.acc_cost = 0
    else:
      self.acc_cost += self.costs[self.s][a]
    self.policy[self.s] = int(a)
    # stocker a dans le vecteur policy. 
    self.V[self.s] = self.Q_Value(a)
    #print("+++++++++++++")
    #print(self.V[self.s])

  def pickNextState(self, a):
    candidates = [i for i in range(0, self.nb_states)]
    return np.random.choice(candidates, p=self.T[a][self.s])

  def residual(self):
    a = self.greedyAction()
    return np.abs(self.Q_Value(a) - self.V[self.s])

  
  
  def checkSolved(self):
    rv = True
    opened = []  #original name is open, but in python open refers to a function
    closed = []
  
    if (self.s not in self.solved):
      opened.append(self.s)
  
    while len(opened) != 0:
      self.s = opened.pop()
      closed.append(self.s)

      if self.residual() > self.epsilon:
        rv = False   
      
      #print("\nCheckSolved")
      #print("residual:")
      #print(self.residual())
      print("state:")
      print(self.s)
      #print("")
      
      a = self.greedyAction()

      for i in range(self.nb_states):
        if self.T[a][self.s][i] != 0:         
          if (i not in self.solved) and (i not in opened + closed):
            opened.append(i)
    #print("closed")
    #print(closed)
    #print("")
        
    if rv:     
      for j in range(len(closed)):
        self.solved.append(closed[j])
        #print(self.solved)
      
    else:
      #print("closed")
      #print(closed)
      #print("")
      while closed != []:
        self.s = closed.pop()
        self.update()

    return rv


  def LRTDP_TRIAL(self):
    visited = []

    while self.s not in self.solved:
      #print("State : " + str(self.s))
      visited.append(self.s)
      #print("visited " + str(visited))

      if self.s == 0:
        self.acc_cost = 0
      
      else:
        self.acc_cost + self.costs[self.s][a]

      if self.costs[self.s][0] >= 0: #Here we can define a goal state as a state where the reward is not negative
        break
        #return self.costs[self.s][0]

      if self.s in self.traps:
        break
        #return self.costs[self.s][0]
      
      a = self.greedyAction()
      self.update()
      print("state 1: " + str(self.s))
      self.s = self.pickNextState(a)
      print("state 2: " + str(self.s))
      #print("\nTRIAL")
      #print("action")
      #print(a)
      #print("##############")
      #print("state")
      #print(self.s)
      #print("##############\n")
      #print("visited " + str(visited))

      while visited != []:
        self.s = visited.pop()
        #print("solved")
        #print(self.solved)
        if not self.checkSolved():
          break
      
  def LRTDP(self):
    #iter = 0
    while self.s not in self.solved:
      #print("\n===============")
      #print(iter)
      #print("===============\n")
      #iter += 1
      #print("state : " + str(self.s))

      #time.sleep(0.1)
      #print("solved")
      #print(self.solved)
      #print("##########")
      self.LRTDP_TRIAL()

    return self.policy, self.V


In [ ]:
print("S3P calculation :")
lrtdp_S3P = mdp(T, C, 1.00, 0.01, False, 1, 0, [1,2,3,9], [14,15,16,17]) # Gamma = 1 !!!
S3P, V_S3P = lrtdp_S3P.LRTDP()

print("\n\n\n")
print("S2P calculation :")
lrtdp_S2P = mdp(T, C, 1.00, 0.01, False, 0, 0, [1,2,3,9], [14,15,16,17]) # Gamma = 1 !!!
S2P, V_S2P = lrtdp_S2P.LRTDP()


Pthd = 0.3
V_S3P0 = V_S3P[0] #Attention ce n'est pas la V-function !!!!
V_S2P0 = V_S2P[0] #Attention !!!
print(V_S3P0)
print(V_S2P0)
PS3P = compute_prob(S3P, 0, T, 1000, [14,15,16,17], False)
PS2P = compute_prob(S2P, 0, T, 10000, [14,15,16,17], False)

print("")
print(PS3P)
print(PS2P)
print()

K = (PS3P * V_S3P0 - Pthd * V_S2P0) / (PS3P - Pthd)

print("\n\nK  = " + str(K))

print("\n\nEP calculation :")
lrtdp_EP = mdp(T, C, 1.00, 0.01, False, 1, K, [1,2,3,9], [14, 15, 16, 17]) # Gamma = 1!!!
EP, V_EP = lrtdp_EP.LRTDP()
PEP = compute_prob(EP, 0, T, 1000, [14,15,16,17],True)



S3P calculation :
state 1: 0
state 2: 1
state:
0
state:
1
state:
14
state:
2
state:
15
state:
3
state:
18
state:
16
state 1: 0
state 2: 4
state:
0
state:
4
state:
8
state:
9
state:
17
state:
10
state:
7
state:
18
state 1: 0
state 2: 4
state:
0
state:
4
state:
8
state:
11
state:
13
state:
12
state:
10
state:
7
state:
18
state 1: 0
state 2: 4
state:
0
state:
4
state:
8
state:
5
state:
9
state:
17
state:
6
state:
10
state:
7
state:
18
state 1: 0
state 2: 1
state:
0
state:
1
state:
14
state:
2
state:
15
state:
6
state:
10
state:
7
state:
18
state 1: 0
state 2: 4
state:
0
state:
4
state:
8
state:
11
state:
13
state:
12
state:
10
state:
7
state:
18




S2P calculation :
state 1: 0
state 2: 1
state:
0
state:
1
state:
14
state:
2
state:
15
state:
3
state:
18
state:
16
state 1: 0
state 2: 4
state:
0
state:
4
state:
8
state:
9
state:
17
state:
10
state:
7
state:
18
state 1: 0
state 2: 1
state:
0
state:
1
state:
14
state:
5
state:
9
state:
17
state:
12
state:
10
state:
7
state:
18
state 1: 0
stat

In [ ]:
print(EP)

[1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 2, 1, 2, 2, 0, 0, 0, 0, 0]


#EP simulation

In [ ]:
policy = [1, 0, 1, 0, 1, 1, 1, 2, 1, 0, 2, 1, 2, 2, 0, 0, 0, 0, 0]
#[1. 2. 2. 2. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
#[1. 2. 2. 0. 1. 2. 1. 2. 1. 0. 2. 1. 2. 2. 0. 0. 0. 0. 0.] S3P
#[0. 0. 0. 0. 1. 1. 2. 2. 0. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0.] S2P
#[1. 0. 1. 0. 1. 1. 1. 2. 1. 0. 2. 1. 2. 2. 0. 0. 0. 0. 0.]


#policy = [1, 1, 0, 0, 1, 0, 0, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0]
nb_states = compute_states(C)
candidates = [i for i in range(0, nb_states)]
print(candidates)
n_episodes = 1000
success = 0
failure = 0
too_slow = 0
for i in range(n_episodes):
  if i%100 == 0: 
    print("episode " + str(i))
  s = 0
  n = 0
  while s < 14:
    n += 1
    s = np.random.choice(candidates, p=T[policy[s]][s])
    
    if n >= 200:
      s = 14
      too_slow += 1
    #print(s)
    if s == 18:
      success += 1
    
    if s > 13 and s < 18 :
      failure += 1
  

print("Failure : " + str(failure))
print("Success : " + str(success))
print("Failure due to slow progression : " + str(too_slow))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
episode 0
episode 100
episode 200
episode 300
episode 400
episode 500
episode 600
episode 700
episode 800
episode 900
Failure : 0
Success : 1000
Failure due to slow progression : 0


#Simulation function

In [ ]:
policy = [1, 0, 1, 0, 1, 1, 1, 2, 1, 0, 2, 1, 2, 2, 0, 0, 0, 0, 0]
#[1. 2. 2. 2. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
#[1. 2. 2. 0. 1. 2. 1. 2. 1. 0. 2. 1. 2. 2. 0. 0. 0. 0. 0.] S3P
#[0. 0. 0. 0. 1. 1. 2. 2. 0. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0.] S2P
#[1. 0. 1. 0. 1. 1. 1. 2. 1. 0. 2. 1. 2. 2. 0. 0. 0. 0. 0.]


#policy = [1, 1, 0, 0, 1, 0, 0, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0]

def compute_prob(policy, initial_state, Transitions, n_episodes, dead_ends, verbosity):
  nb_states = compute_states(policy)
  candidates = [i for i in range(0, nb_states)]
  success = 0
  failure = 0
  too_slow = 0
  for i in range(n_episodes):
    if i%100 == 0 and verbosity: 
      print("episode " + str(i))
    s = initial_state
    n = 0
    while s not in dead_ends and s != nb_states - 1:
      n += 1
      s = np.random.choice(candidates, p=Transitions[policy[s]][s])
    
      if n >= 200:
        s = dead_ends[0]
        too_slow += 1
      #print(s)
      if s == nb_states - 1:
        success += 1
    
      if s in dead_ends :
        failure += 1
  
  if verbosity:
    print("Failure : " + str(failure))
    print("Success : " + str(success))
    print("Failure due to slow progression : " + str(too_slow))
    print("")

  return success/n_episodes